# Factory
Proporciona una interfaz para crear objetos en una superclase, mientras permite a las subclases alterar el tipo de objetos que se crearán.

## Caso de estudio: Sistema de preparacion de comidas rápidas

Se dispone de un sistema de preparacion de comidas rápidas, las clases "comida" se encargan de gestionar el proceso de la comida mientras que las clases fábrica se encargan de crearlas/instanciarlas, el modelo presentado corresponde a un factory method

In [3]:
from typing import List
from abc import ABC, abstractmethod

# Interface factory
class IFoodFactory(ABC):
    @abstractmethod
    def create_food(self):
        pass

class PizzaFactory(IFoodFactory):
    def create_food(self):
        return Pizza()
    
class BurgerFactory(IFoodFactory):
    def create_food(self):
        return Burger()

# Interface product
class IFood(ABC):
    @abstractmethod
    def prepare(self):
        pass

    @abstractmethod
    def serve(self):
        pass

class Pizza(IFood):
    def prepare(self):
        print("Pizza is preparing...")

    def serve(self):
        print("Pizza is serving...")

class Burger(IFood):
    def prepare(self):
        print("Burger is preparing...")

    def serve(self):
        print("Burger is serving...")

class Client:
    def __init__(self, food_factory: IFoodFactory):
        self.food_factory = food_factory
    
    def execute(self):
        food = self.food_factory.create_food()
        food.prepare()
        food.serve()

if __name__ == "__main__":
    client1 = Client(PizzaFactory())
    client1.execute()
    
    client2 = Client(BurgerFactory())
    client2.execute()

Pizza is preparing...
Pizza is serving...
Burger is preparing...
Burger is serving...


El modelo planteado satisface los principios SOLID:

    S - Principio de Responsabilidad Única (SRP)
        Cada clase tiene una única responsabilidad:
        Pizza y Burger manejan la preparación y el servicio de la comida.
        PizzaFactory y BurgerFactory se encargan de la creación de instancias de Pizza y Burger.
        client se encarga de la lógica de ejecución sin conocer los detalles de los objetos creados.
    
    O - Principio de Abierto/Cerrado (OCP)
        Se pueden agregar nuevos tipos de comida sin modificar las clases existentes, simplemente creando nuevas clases que implementen IFood y IFoodFactory.
    
    L - Principio de Sustitución de Liskov (LSP)
        Pizza y Burger pueden ser utilizadas donde se espera un IFood, y PizzaFactory y BurgerFactory pueden sustituir a IFoodFactory sin afectar el comportamiento del código.
    
    I - Principio de Segregación de Interfaces (ISP)
        Las interfaces IFood y IFoodFactory aseguran que las clases solo implementen los métodos que realmente necesitan.
    
    D - Principio de Inversión de Dependencias (DIP)
    client no depende de implementaciones concretas, sino de las abstracciones IFoodFactory y IFood.

Adicionalmente se satisfacen los siguientes principios GRASP:

    1 - Creator
        PizzaFactory y BurgerFactory siguen el patrón Factory Method y son responsables de crear instancias de Pizza y Burger.

    2 - Polimorfismo
        Se utiliza polimorfismo al definir métodos abstractos en IFood y IFoodFactory, permitiendo que Pizza y Burger los implementen sin que el código cliente necesite conocer los detalles.

    3 - Indirection
        Se usa la fábrica como intermediario entre el cliente y la creación de objetos concretos, reduciendo el acoplamiento.

    4 - Low Coupling
        client no depende de implementaciones específicas, sino de interfaces, lo que facilita la extensibilidad y el mantenimiento del código.

    5 - High Cohesion
        Cada clase tiene una única razón para cambiar y está enfocada en una tarea específica, lo que mejora la cohesión.

# Multiton
Multiton es una extensión del patrón Singleton, donde en lugar de tener una única instancia global, se gestionan múltiples instancias identificadas y localizables mediante una variable de clave/valor.

El patron multiton se puede implementar en una aplicación que realiza conexiones a diferentes bases de datos, en este caso crearemos una clase que instanciará diferentes singleton asociados a la conexion de una base de datos diferente para que ésta conexion no se produzca múltiples veces cada vez que se instancia la conexion

In [10]:
class DatabaseConnection:
    __instances = {} #Creamos un diccionario para almacenar los singleton

    def __new__(cls, db_name):
        if db_name not in cls.__instances:
            instance = super().__new__(cls)
            instance.db_name = db_name
            instance.id = hash(db_name)
            print(f"Creating new instance for {db_name}")
            cls.__instances[db_name] = instance
        return cls.__instances[db_name]
    
    def query(self, sql):
        print(f"Executing query {sql} in {self.db_name}")

if __name__ == "__main__":
    # Creamos 3 instancias de la clase DatabaseConnection
    db1 = DatabaseConnection("mysql")
    db2 = DatabaseConnection("PGsql")
    db3 = DatabaseConnection("sqlite\n")

    # Validamos la funcionalidad
    db1.query("SELECT * FROM users")
    db2.query("SELECT * FROM products")
    db3.query("SELECT * FROM store\n")

    # Creamos otra instancia de una conexion a mysql
    db4 = DatabaseConnection("mysql")
    db4.query("SELECT * FROM products\n")
    
    # Validamos que las instancias sean las mismas
    print(db1.id)
    print(db4.id)

Creating new instance for mysql
Creating new instance for PGsql
Creating new instance for sqlite

Executing query SELECT * FROM users in mysql
Executing query SELECT * FROM products in PGsql
Executing query SELECT * FROM store
 in sqlite

Executing query SELECT * FROM products
 in mysql
5162417358122147043
5162417358122147043


El patron multiton cumple algunos principios SOLID

    S - Principio de Responsabilidad Única (SRP)
        Cumple parcialmente: La clase DatabaseConnection se encarga de administrar conexiones a bases de datos y ejecutar consultas. Sin embargo, también gestiona el almacenamiento instancias, lo que representa una violación leve. Idealmente, la gestión de conexiones debería estar en una clase separada, como ConnectionManager.

    O - Principio de Abierto/Cerrado (OCP)
        Viola: Si queremos agregar soporte para conexiones con autenticación, pooling u otro tipo de configuración, tendríamos que modificar la clase actual en lugar de extender su funcionalidad sin modificarla.

    L - Principio de Sustitución de Liskov (LSP)
        Cumple: No hay herencia en este código, por lo que no se está violando el LSP.

    I - Principio de Segregación de Interfaces (ISP)
        Cumple: La clase no implementa interfaces grandes ni métodos innecesarios.

    D - Principio de Inversión de Dependencias (DIP)
        Viola: La clase DatabaseConnection depende directamente de la estructura de su propio diccionario de instancias. No está abstraída mediante una interfaz o inyección de dependencias.

El patron multiton cumple algunos principios GRASP

    1 - Information Expert (Experto en Información)
        Cumple: La clase que maneja las conexiones es quien tiene la información sobre ellas.
    
    2 - Creator (Creador)
        Cumple: La clase crea y maneja instancias de sí misma correctamente.
    
    3 - High Cohesion (Alta Cohesión)
        Parcialmente: La clase mezcla la gestión de conexiones y la ejecución de consultas, lo que disminuye un poco la cohesión.
    
    4 - Low Coupling (Bajo Acoplamiento)
        Viola: La clase depende directamente de su implementación interna. No hay abstracción de las conexiones.
    
    5 - Singleton (Usado como Multiton)
        Cumple: La clase usa una variante del patrón Singleton para gestionar múltiples conexiones únicas por base de datos.